In [1]:
import numpy as np 
import pandas as pd 
import os
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from bayes_opt import BayesianOptimization

import pandas as pd
import numpy as np 
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier


##### DON'T FORGET TO SET A MAX NUMBER OF ITERATIONS HERE. DON'T GO WITH -1 PLEASE
from sklearn.svm import SVC


/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/michael/anaconda3/lib/python3.6/site-packages/tensorflow/python/frame

In [2]:
df = pd.read_csv("creditcard.csv")

df = df.drop(['Time','V28','V27','V26','V25','V24','V23','V22','V20','V15','V13','V8'], axis =1)
df = df.dropna()


train_df = df.sample(frac=0.8, random_state=0)
test_df =  df.drop(train_df.index)

train_df = shuffle(train_df)
test_df = shuffle(test_df)

y_train = train_df.pop('Class').as_matrix()
y_test = test_df.pop('Class').as_matrix()
X_train = train_df.as_matrix()
X_test = test_df.as_matrix()



#X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)

In [3]:
# new block!
# Let's try to make some functions that will help us parse the config
from config import *
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# a function to create the pbounds dictionary for each model
def make_pbounds():
    
    # initialize an empty dictionary for all the pbounds of the models to go into
    pbounds_dict = {}
    
    # we should already have global variables for the config things
    # run through the models in model_config cause those are the ones we're actually using
    for model_name in model_config.keys():
        
        # initialize the model's pbounds dict
        model_pbounds = {}
        
        # check if the hparam's dictionary is empty. If so, we need to keep the pbounds dict empty
        ##### IF THE PBOUNDS DICT IS EMPTY, IT MEANS THE MODEL IS NOT TO BE OPTIMIZED, JUST TRAINED
        if model_config[model_name]['hyperparameters'] == {}:
            pbounds_dict[model_name] = model_pbounds
            continue
        
        # and go through all the hyperparameters in here
        for hparam_name in model_config[model_name]['hyperparameters'].keys():
            
            ##### WE WILL NEED SUPPORT TO ADD THE ALIAS HERE AND REMOVE IT AT THE COMPLETION OF OPTIMIZATION
            ##### WHEN WE ALLOW MULTIPLE MODELS FROM EACH TYPE
            
            
            
            # if 'range' is 'base', dig into the classification_models dict
            if model_config[model_name]['hyperparameters'][hparam_name]['range'] == 'base':
                
                model_pbounds[hparam_name] = classification_models[model_name]['hyperparameters'][hparam_name]['default_range']
            
            # else we're good to use this one
            else:
                model_pbounds[hparam_name] = model_config[model_name]['hyperparameters'][hparam_name]['range']
        
        # now we should have a completed model_pbounds, add it to pbounds_dict
        pbounds_dict[model_name] = model_pbounds
    
    # then return the pbounds_dict!
    return pbounds_dict

# function to import all the models
##### I have nooo idea if this will do it globally or if we'll need some help
def import_models():
    
    # loop through all the models in model_config and call their import statements written in classification_models
    for model_name in model_config.keys():
        import_statement = classification_models[model_name]['import_statement']
        
        exec(import_statement, globals())

# a function to determine which kind of error we'll be optimizing for
def error(preds, y_test):
    
    error_name = config['metric']
    
    if error_name == 'roc_auc':
        return roc_auc_score(preds, y_test)
    elif error_name == 'accuracy':
        return accuracy_score(preds, y_test)
    elif error_name == 'recall':
        return recall_score(preds, y_test)
    elif error_name == 'precision':
        return precision_score(preds, y_test)
    elif error_name == 'f1':
        return f1_score(preds, y_test)
    else:
        print('unsure what your metric is in the config so using accuracy instead')
        return accuracy_score(preds, y_test)
    
    
# cool now we need another function that will create the model
# this is tricky but I think we should bite the bullet with having to search up things in config every time it's called
# the majority of the time this function is called will be in the actual model fitting anyway
@ignore_warnings(category=ConvergenceWarning)
def error_function_base(**hparams):
    
    
    #print('model name in base error function: ', model_name)
    
    
    # so in this we're just going to have the optimization kwargs
    ### we also need a model name!
    # let's pretend a global model_name has been created for this before we enter this function and we can use it
    
    
    # we need to go through each hparam in hparams and find out which ones need to be made into an int!
    for hparam_name in hparams.keys():
        
        if classification_models[model_name]['hyperparameters'][hparam_name]['type'] == 'int':
            hparams[hparam_name] = int(hparams[hparam_name])
    
    # create an error function based on the inputs
    ##### DON'T FORGET A LOT OF THESE CAN USE -1 CORES
    
    # then create the model
    # I surely hope we can do this in string format. May need to play around with global-ness
    # may also need some asterisks
    
    # ok this is messy we'll just put this into a list
    temp_list = []
    model_str = 'temp_list.append({}(**hparams))'.format(model_name)
    exec(model_str)
    
    model = temp_list[0]
    
    total_error = 0
    
    for i in range(config['num_sub_its']):
        
        # every time we're going to get a new sample
        train_df = df.sample(frac=0.8, random_state=0)
        test_df =  df.drop(train_df.index)
        train_df = shuffle(train_df)
        test_df = shuffle(test_df)
        y_train = np.nan_to_num(train_df.pop('Class').as_matrix())
        y_test = np.nan_to_num(test_df.pop('Class').as_matrix())
        X_train = np.nan_to_num(train_df.as_matrix())
        X_test = np.nan_to_num(test_df.as_matrix())
        
        # fit the model
        try:
            model.fit(X_train, y_train)
        except ValueError:
            continue
        preds = model.predict(X_test)

        try:
            temp_error = error(preds, y_test)
        except:
            temp_error = 0
        
        total_error += temp_error
    
    
    return total_error/config['num_sub_its']

# a function that runs the optimization of each of the base models and also stores their results
def run_base_optimization():
    
    # we need to import the models
    import_models()
    
    # then make the pbounds dictionary
    pbounds_dict = make_pbounds()
    
    # create a dictionary to store the results
    # key:model value:hparam dict of maximum
    hparam_max_dict = {}
    
    # and a dictionary to store the actual models themselves?
    # oh nuts will we have to fit the model after optimization? Probably.
    model_store_dict = {}
    
    # and now run through the optimization of the models!
    for model_name in model_config.keys():
        
        print('Optimizing ' + model_name)
        
        exec('model_name = \'{}\''.format(model_name), globals())
        
        
        # get the pbounds
        model_pbounds = pbounds_dict[model_name]
        
        ### this is where we should check to see if it's empty
        if model_pbounds == {}:
            
            # then we should just train the model, add it, and continue
            temp_list = []
            model_str = 'temp_list.append({}())'.format(model_name)
            exec(model_str)
            model = temp_list[0]
            model.fit(X_train, y_train)
            continue
        
        #print('model name in run_optimization: ' + model_name)
        
        optimizer = BayesianOptimization(
            f=error_function_base,
            pbounds=model_pbounds)
        optimizer.maximize(init_points=2, n_iter=config['num_model_its'], xi=0.5)
        max_hparams = optimizer.max
        hparam_max_dict[model_name] = max_hparams
        
        ### YOU NEED TO RE-INT ALL THE MAX HPARAMS
        for hparam_name in max_hparams['params'].keys():
            
            # skip if it's 'target'
            if classification_models[model_name]['hyperparameters'][hparam_name]['type'] == 'int':
                max_hparams['params'][hparam_name] = int(max_hparams['params'][hparam_name])
        
        # then refit the model with these parameters
        temp_list = []
        model_str = 'temp_list.append({}(**max_hparams[\'params\']))'.format(model_name)
        exec(model_str)
        model = temp_list[0]
        model.fit(X_train, y_train) # hopefully these can be made global parameters? Otherwise we need help
        
        # and save these
        model_store_dict[model_name] = model
    
    # return the hparam and model dictionaries
    return hparam_max_dict, model_store_dict

# create the dataset for the metalearner based on the perviously trained models
def create_meta_dataset(model_store_dict):
    
    train_df = df.sample(frac=0.8, random_state=0)
    test_df =  df.drop(train_df.index)
    train_df = shuffle(train_df)
    test_df = shuffle(test_df)
    y_train = np.nan_to_num(train_df.pop('Class').as_matrix())
    y_test = np.nan_to_num(test_df.pop('Class').as_matrix())
    X_train = np.nan_to_num(train_df.as_matrix())
    X_test = np.nan_to_num(test_df.as_matrix())
        
    # loop through the models
    for model_name in model_store_dict.keys():
        
        # grab the np array of the predictions on our training df
        preds_np_train = model_store_dict[model_name].predict(X_train)
        preds_np_test = model_store_dict[model_name].predict(X_test)
        
        # make this a series
        preds_series_train = pd.Series(preds_np_train)
        preds_series_test = pd.Series(preds_np_test)
        
        # assign to the big df
        exec('train_df.assign(preds_{}=preds_series_train)'.format(model_name))
        exec('test_df.assign(preds_{}=preds_series_test)'.format(model_name))
    
    # this should already be good-to-go in train and test dfs
    #print(train_df.columns)
    
    
    
# yikes so now how do you do the ensemble optimization?
# for the moment what if we just create a stacked ensemble and only have one meta-learner?
# optimize the hyperparameters on that one
# create the error function
def error_function_meta(**hparams):
    
    model_name = 'ExtraTreesClassifier'
    
    # we need to go through each hparam in hparams and find out which ones need to be made into an int!
    for hparam_name in hparams.keys():

        if classification_models[model_name]['hyperparameters'][hparam_name]['type'] == 'int':
            hparams[hparam_name] = int(hparams[hparam_name])

    
    # then create the model
    # I surely hope we can do this in string format. May need to play around with global-ness
    # may also need some asterisks
    temp_list = []
    model_str = 'temp_list.append({}(**hparams))'.format(model_name)
    exec(model_str)
    model = temp_list[0]
    
    #X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)

    # every time we're going to get a new sample
    #train_df = df.sample(frac=0.8, random_state=0)
    #test_df =  df.drop(train_df.index)
    #train_df = shuffle(train_df)
    #test_df = shuffle(test_df)
    #y_train = np.nan_to_num(train_df.as_matrix())
    #y_test = np.nan_to_num(test_df.pop('Class').as_matrix())
    #X_train = np.nan_to_num(train_df.as_matrix())
    #X_test = np.nan_to_num(test_df.as_matrix())

    # fit the model
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    try:
        temp_error = error(preds, y_test)
    except:
        temp_error = 0

    return temp_error

# then the meta learner
def run_meta_optimization(hparam_max_dict):
    
    
    # then make the pbounds dictionary
    pbounds_dict = make_pbounds()
    
    # and now run through the optimization of the models!
    print('Optimizing meta learner')

    # get the pbounds
    model_pbounds = pbounds_dict['ExtraTreesClassifier']

    optimizer = BayesianOptimization(
        f=error_function_meta,
        pbounds=model_pbounds)
    optimizer.maximize(init_points=2, n_iter=config['meta_learner_its'], xi=0.5)
    max_hparams = optimizer.max
    
    print()
    print('Finished optimization')
    print('Max model metrics:')
    for model_name in hparam_max_dict.keys():
        
        # pull out the target
        print(' {}: {}'.format(model_name, hparam_max_dict[model_name]['target']))
    print('Meta-learner: {}'.format(max_hparams['target']))
    
def run():
    
    hparam_max_dict, model_store_dict = run_base_optimization()
    create_meta_dataset(model_store_dict)
    run_meta_optimization(hparam_max_dict)
    

/Users/michael/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
from config import *
run()

Optimizing LogisticRegression
|   iter    |  target   |     C     |    tol    |
-------------------------------------------------
|  1        |  0.6777   |  3.559    |  0.006143 |
|  2        |  0.6914   |  4.667    |  0.001229 |
|  3        |  0.684    |  10.0     |  0.01     |
|  4        |  0.672    |  7.371    |  0.01     |
|  5        |  0.0      |  1e-05    |  1e-06    |
|  6        |  0.7036   |  8.866    |  1e-06    |
|  7        |  0.6848   |  9.314    |  0.01     |


/Users/michael/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Optimizing GaussianNB
Optimizing ExtraTreesClassifier
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.2325   |  2.234    |  281.6    |
|  2        |  0.7508   |  9.592    |  38.69    |
|  3        |  0.7771   |  10.0     |  500.0    |
|  4        |  0.6567   |  8.458    |  498.2    |
|  5        |  0.2274   |  2.03     |  104.1    |
|  6        |  0.784    |  10.0     |  382.6    |
|  7        |  0.7747   |  10.0     |  193.1    |
Optimizing AdaBoostClassifier
|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
|  1        |  0.001193 |  4.37     |  450.5    |
|  2        |  0.001193 |  4.159    |  258.7    |
|  3        |  0.00354  |  8.517    |  10.01    |
|  4        |  0.6946   |  0.07368  |  10.08    |
|  5        |  0.5828   |  0.01     |  33.42    |
|  6        |  0.6946   |  0.01     |  65.83    |
|  7        |  0.0      |  10.0     |  95.9     |
Optimizing meta 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn
model2 = knn(n_neighbors = 1)
model4 = knn(n_neighbors = 2)
model8 = knn(n_neighbors = 3)
model16 = knn(n_neighbors = 4)
model32 = knn(n_neighbors = 5)
model64 = knn(n_neighbors = 6)
model128 = knn(n_neighbors = 7)
model256 = knn(n_neighbors = 8)

model_list = [model2, model4, model8, model16, model32, model64, model128, model256]
ens_list = list(zip(['1','2','3','4','5','6','7','8'], model_list))

i = 0
for model in model_list:
    
    i += 1
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #acc_err = accuracy_score(preds, y_test)
    roc_err = roc_auc_score(preds, y_test)
    recall_err = recall_score(preds, y_test)
    prec_err = precision_score(preds, y_test)
    #f1_err = f1_score(preds, y_test)
    
    print(2**i)
    #print(' accuracy: {}'.format(acc_err))
    print(' roc: {}'.format(roc_err))
    print(' recall: {}'.format(recall_err))
    print(' precision: {}'.format(prec_err))
    #print(' f1: {}'.format(f1_err))
    print()

# train the stacker
stack_class = StackingClassifier(ens_list)
stack_class.fit(X_train, y_train)
preds = stack_class.predict(X_test)
#acc_err = accuracy_score(preds, y_test)
roc_err = roc_auc_score(preds, y_test)
recall_err = recall_score(preds, y_test)
prec_err = precision_score(preds, y_test)
#f1_err = f1_score(preds, y_test)
print('Stacked ensemble:')
#print(' accuracy: {}'.format(acc_err))
print(' roc: {}'.format(roc_err))
print(' recall: {}'.format(recall_err))
print(' precision: {}'.format(prec_err))
#print(' f1: {}'.format(f1_err))

In [ ]:
sh_list = ['soft', 'hard']
model_list_slices = [[0,1], [0,2], [0,3], [1,2], [1,3], [2,3], [0,1,2], [0,1,3], [0,2,3], [1,2,3], [0,1,2,3]]

In [ ]:
def error_function(**kwargs):#lr, n_est, md, msf, c, ada_n_est, nn, model_list_index, sh_index):
    
    print(kwargs)
    sh_index = int(sh_index)
    model_list_index = int(model_list_index)
    msf = int(msf)
    md = int(md)
    n_est = int(n_est)
    ada_n_est = int(ada_n_est)
    nn = int(nn)
    
    sh_choice = sh_list[sh_index]
    model_slice = model_list_slices[model_list_index]
    
    # create an error function based on the inputs
    ##### DON'T FORGET A LOT OF THESE CAN USE -1 CORES
    xgb_model = xgb(learning_rate=lr, n_estimators=n_est, max_depth=md, min_samples_leaf=msf)
    lr_model = LogisticRegression(C=c, solver='lbfgs')
    knn_model = KNeighborsClassifier(n_neighbors=nn)
    ada = AdaBoostClassifier(n_estimators=ada_n_est)
    
    models = [('1', xgb_model), ('2', lr_model), ('3', knn_model), ('4', ada)]
    
    model_choice = [models[n] for n in model_slice]
    
    
    
    total_error = 0
    
    for i in range(1):
        #X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, train_size=0.75, test_size=0.25)
        
        ens = VotingClassifier(model_choice, voting=sh_choice)
        ens.fit(X_train, y_train)
        preds = ens.predict(X_test)
        
        try:
            error = roc_auc_score(preds, y_test)
        except:
            error = 0
        
        total_error += error
    
    error = total_error
    
    return error

In [ ]:
pbounds = {'lr': (0.00001, 1), 
           'n_est': (1, 10), 
           'md': (1,100), 
           'msf': (1, 10),
           'c': (0.01, 10),
           'ada_n_est': (1, 100),
           'nn': (1, 100),
           'sh_index': (0, 1),
           'model_list_index': (0, len(model_list_slices) - 1)}

optimizer = BayesianOptimization(
    f=error_function,
    pbounds=pbounds
)

In [ ]:
optimizer.maximize(init_points=2, n_iter=1, xi=0.5)

In [ ]:
print(optimizer.max)